## Import Libs and Setup Params

In [4]:
!pip install bitsandbytes
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

/opt/miniconda3/envs/uni311/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
def get_prompt(prompt_name:str) -> str:
    prompt = ""
    match prompt_name:
        case "in_context":
            prompt = """Sei un esperto di lingua italiana. Il tuo compito è modernizzare testi antichi in italiano contemporaneo. Di seguito alcuni esempi:

            Testo antico: quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
            Traduzione moderna: Quella guerra fu ben condotta per raggiungere il suo scopo. Dall'altra parte, Aiace era un cavaliere leale e valoroso nelle armi, di grande statura, ma non molto saggio.

            Testo antico: crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
            Traduzione moderna: È crudele e si vendica di ogni colpa, come stabilisce la legge, e non perdona alcun cavaliere che commetta un errore.

            Testo antico: Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.
            Traduzione moderna: Ponzio Aufidiano, cavaliere romano, non era dotato di un coraggio superiore.

            Testo antico: Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati.
            Traduzione moderna: Se questo è il volere di tutti, e se i tempi richiedono Pompeo come guida e non come compagno, allora non tratterrò oltre il destino.

            Testo antico: Officio di questa arte pare che sia dicere appostatamente per fare credere, fine è far credere per lo dire.
            Traduzione moderna: Il compito di quest’arte sembra essere quello di parlare in modo studiato per convincere; il fine è dunque persuadere attraverso le parole.

            Perfavore non includere nella risposta il testo precedente, ma soltanto la traduzione.
            Traduci la frase da italiano antico a quello moderno, scrivendo solo ed ESCLUSIVAMENTE la risposta:
                {sentence}
            """
        case "language_expert":
            prompt = """Sei un esperto di lingua italiana. Il tuo compito è modernizzare testi antichi in italiano contemporaneo.

            Perfavore non includere nella risposta il testo precedente, ma soltanto la traduzione.
            Traduci la seguente frase da italiano antico a quello moderno, scrivendo solo ed ESCLUSIVAMENTE la risposta:
                {sentence}
            """
        case "with_rules":
            prompt = """Traduci e modernizzare una frase in italiano contemporaneo e comprensibile.

            Devi seguire le seguenti regole:
                1. La frase prodotta deve essere comprensibile
                2. La frase prodotta deve avere un senso
                3. Tutte le parole devono essere in italiano contemporaneo, escludendo nomi o luoghi
            
            Perfavore restuituisci SOLO ED ESCLUSIVAMENTE la la frase tradotta e modrnizzata.

            La frase da tradurre è la seguente:
                {sentence}
            
            Output:
            """
        case _:
            raise Exception("Unexpected name")
            exit(1)

    return prompt
        

In [11]:
quantize_model = True

"""
You can choose from:
    1. "in_context"
    2. "language_expert"
    3. "with_rules"


"""
prompt = get_prompt("with_rules")
prompt

'Traduci e modernizzare una frase in italiano contemporaneo e comprensibile.\n\n            Devi seguire le seguenti regole:\n                1. La frase prodotta deve essere comprensibile\n                2. La frase prodotta deve avere un senso\n                3. Tutte le parole devono essere in italiano contemporaneo, escludendo nomi o luoghi\n\n            Perfavore restuituisci SOLO ED ESCLUSIVAMENTE la la frase tradotta e modrnizzata.\n\n            La frase da tradurre è la seguente:\n                {sentence}\n\n            Output:\n            '

In [ ]:
from transformers import BitsAndBytesConfig

quant_config = None
if quantize_model:
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype="float16",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )

## Load Test Dataset

In [ ]:
df = pd.read_csv("test.csv", sep=";")
df = df.rename(columns={"Sentence": "source", "Traductions": "target"})

dataset = Dataset.from_pandas(df)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## Define Quantization and Load Model

In [ ]:
model_name = "google/flan-t5-XL"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name,
                                              device_map="auto",
                                              quantization_config=quant_config).to(device)

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


## Prometheus Evaluator

In [ ]:
from prometheus import PrometheusEval_AtM

evaluator = PrometheusEval_AtM(quantized = True, device = device)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
def translate_with_flan(prompt,sentence):
    prompt = f"{prompt}: {sentence}"
    return pipe(prompt, max_new_tokens=300)[0]['generated_text']

In [ ]:
import re
i = 1

source_sentences = []
predicted_sentences = []
gold_sentences = [] #annoted by hand
prometheus_score = []
df_col = ["source_sentences", "gold_sentences","predicted_sentences", "prometheus_score","GPT_score","user_score"]

print(len(dataset))
for sample in dataset:


    input_sentence = sample["source"]
    target_sentence = sample["target"]

    author = sample["Author"]
    date = sample["Date"]
    region = sample["Region"]

    user_prompt = prompt.format(sentence=input_sentence)

    translation = translate_with_flan(user_prompt,input_sentence)

    evaluation = evaluator.getEvaluation(input_sentence, translation, target_sentence)


    match_ = re.search(r'\[RESULT\]\s*(\d)', evaluation)
    if match_:
      result = int(match_.group(1))

    else:
      result = 0

    source_sentences.append(input_sentence)
    predicted_sentences.append(translation)
    gold_sentences.append(target_sentence)
    prometheus_score.append(result)



    print(f"Sentence {i}")
    print(f"\tItaliano Arcaico            -> {input_sentence}")
    print(f"\tItaliano moderno            -> {translation}")
    print(f"\tGOLD LABEL                  -> {target_sentence}")
    print(f"\tPROMETHEUS EVALUATION       -> {result}")
    print(f"-----------------------------------------")
    i+=1

z = [0 for _ in range(len(dataset))]
GPT_score,user_score = z,z
df = pd.DataFrame(list(zip(source_sentences, gold_sentences,predicted_sentences,prometheus_score,GPT_score,user_score)), columns=df_col)

df.to_csv("test_results_base.csv", sep=";")

10


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sentence 1
	Italiano Arcaico            -> Et se l' occhio è nobile membro del corpo dell' uomo, dunque la salutazione è nobile parte della pistola, c' altressì allumina tutta la lettera come l' occhio allumina l' uomo.
	Italiano moderno            -> Se l'occhio è nobile membro del corpo dell'uomo, dunque la salutazione è nobile parte della pistola, c' altress alluminata tutta la lettera come l'occhio allumina l'uomo.
	GOLD LABEL                  -> E se l’occhio è una parte nobile del corpo umano, allora il saluto è una parte nobile della lettera, che illumina l’intero testo come l’occhio illumina l’uomo.
	PROMETHEUS EVALUATION       -> 4
-----------------------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sentence 2
	Italiano Arcaico            -> Tarentini, i quali erano nati di quegli di Lacedemonia et facta da lloro nobile cittade de' Greci.
	Italiano moderno            -> Traduci la frase da italiano antico a quello moderno, scrivendo solo ed ESCLUSIVAMENTE la risposta: : Tarentini, i quali erano nati di questi di Lacedemonia et facta da lloro nobile cittade de' Greci.
	GOLD LABEL                  -> I Tarantini, nati dai lacedemoni, avevano fondato la loro nobile città greca.
	PROMETHEUS EVALUATION       -> 1
-----------------------------------------


...

Sentence 3
	Italiano Arcaico            -> Ulecois, ebe un uomo rico e nobile: Orgentore fue chiamato per nome.
	Italiano moderno            -> Sei un esperto di lingua italiana. Il tuo compito è modernizzare testi antichi in italiano contemporaneo. Di seguito alcuni esempi: Testo antico: quella guerra ben fatta l' opera perché etc. Et dall'altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno Traduzione moderna: Quella guerra fu ben condotta per raggiungere il suo scopo. Dall'altra parte, Aiace era un cavaliere leale e valoroso nelle armi, di grande statura, ma non molto saggio. Testo antico: Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere. Traduzione moderna: Ponzio Aufidiano, cavaliere romano, non era dotato di un coraggio superiore. Testo antico: Se questo piace a tutti e se 
	GOLD LABEL                  -> Ulecois ebbe un uomo ricco e nobile: si chiamava Orgentore.
	PROMETHEUS EVALUATION       -> 1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sentence 4
	Italiano Arcaico            -> però che, sse nobile cosa e alta è abatte il nimico, ampoi nonn è meno laudabile sapere avere misiricordia
	Italiano moderno            -> Sei un esperto di lingua italiana. Il tuo compito è modernizzare testi antichi in italiano contemporaneo. Di seguito alcuni esempi: Testo antico: quella guerra ben fatta l'opera perché etc. Et dall'altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno Traduzione moderna: Quella guerra fu ben condotta per raggiungere il suo scopo. Dall'altra parte, Aiace era un cavaliere leale e valoroso nelle armi, di grande statura, ma non molto saggio. Testo antico: Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterr più i fati. Traduzione moderna: Se questo è il volere di tutti, e se i tempi rich
	GOLD LABEL                  -> Perché se è una cosa alta e nobile abbattere il nemico, è altrettanto lodevole s

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 